In [1]:

import pandas as pd
import numpy as np
import math


class NWKNN:
    Data = []
    """
    Neighbor-weighted K-Nearest Neighbors
    init adalah constructor
    """
    def __init__(self, n_neighbors=0, exp=0):
        if n_neighbors < 1:
            raise Exception('n_neighbors must be greater than 0')
        if exp <= 1:
            raise Exception('exp must be greater than 1')
            
        self.n_neighbors = n_neighbors
        self.exp = exp
        self.X_train = None
        self.y_train = None
        self.weights = None
        self.distance = None
        #self.dataset_df = None

    def __calculate_weight(self):
        #counts = dict(pd.value_counts(self.dataset_df))
        counts = dict(pd.value_counts(self.y_train))
        #min_count = counts[min(counts)]
        
        self.weights = dict()
        #print(" \n")
        sortedDict = sorted(counts.items(),key=lambda x:x[1])
        terendah = sortedDict[0][1]
        #print(" jumlah terendah : {}".format(terendah))
        
        for kelas in counts:
            divide_result = counts[kelas] / terendah
            power = 1 / self.exp
            lower_result = pow(divide_result, power)
            self.weights[kelas] = 1 / lower_result
            #print(" Kelas : {}".format(kelas))
            #print(" divide_result : {}".format(divide_result))
            #print(" lower_result : {}".format(lower_result))
            #print(" weight : {}".format(self.weights[kelas]))
            #print("=================")
            
            #print(self.weights) #nilai Weight


    def fit(self, X_train, y_train):
        #print("fit called")
        self.X_train = X_train
        self.y_train = y_train
        self.__calculate_weight()
        #self.dataset_df = dataset_df
#         print(X_train)
#         print(y_train)
        
    
    def __decision_func(self, distances):
        distances_df = pd.DataFrame(distances, columns=['distance', 'class'])
        distances_df = distances_df.sort_values(by=['distance'])
        #print(distances_df.head(self.n_neighbors)) #nilai ED
         
        neighbors = distances_df.head(self.n_neighbors)

        # ipakai weight
        prediction = self.__calculate_score(neighbors)

        # tidak pakai weight
        #prediction = neighbors['class'].value_counts().idxmax()
        return prediction



    def __calculate_score(self, neighbors):
        #print(" __calculate_score called")
        #print("neighbor", neighbors)
        result = []
        score_result = []

        keys = ['Tinggi', 'Rendah', 'Sedang']
        scores = []
        for index, item in neighbors.iterrows():
            temp_list = []
            temp = {
            }
            if item['distance'] != 0:
                temp[item['class']] = item['distance']
            temp_list.append(temp)
            scores.append(temp)
        
        temp = {
            "Tinggi":0,
            "Rendah":0,
            "Sedang":0
        }
        for key in keys:
            temp_result = 0
            for x in scores:
                if key in x:
                    temp_result = temp_result + x[key]
            temp[key] = temp_result
        score_result = temp

        result = ''
        for key in score_result:
            if score_result[key] == max(score_result.values()):
                result = key
        #print(score_result)
        return result
    
        # nilai weight  self.weights (bentuk dictionary)
        # check value print()
        # return kelas hasil nya
        

    def predict(self, X_test):
        #print("predict called")
        data = []
        y_pred = []
        num = 0
        eucli_df = pd.DataFrame()
        
        for i, test_row in X_test.iterrows():
            #print("matrix data ke", num)
            kolom = []
            num +=1
            distances = []
            
            l= 0
            for j, train_row in self.X_train.iterrows():
              
                distance = np.linalg.norm(test_row - train_row) 
                kelas = y.iloc[j]
                distances.append((distance, kelas))
                #print("hasil {0}: {1}".format(l,distance))
                kolom.append(distance)
                l+=1
            data.append(kolom)
            prediction = self.__decision_func(distances)
            y_pred.append([prediction])
        self.Data.append(data)   
        return pd.Series((v[0] for v in y_pred))
    


In [2]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample

#dataset_df = pd.read_csv('dataset_fix_1000.csv')
#dataset_df = pd.read_csv('dataset_fix.csv')
dataset_df = pd.read_csv('dataset_terbaru_fix.csv')

# Menambahkan duplikat data sebanyak 750 data
#dataset_df = resample(dataset_df, replace=True, n_samples=1000, random_state=0)

X = dataset_df.iloc[:, :22] 
y = dataset_df['kelas']


list_k = [20]
list_exp = [4]

best_k = 0
best_exp = 0
best_avg_acc = 0


In [3]:
#print(":", dataset_df.shape[0])

In [5]:
skf = StratifiedKFold(n_splits=10, random_state=42,shuffle=True)
# skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
for k in list_k:
    for exp in list_exp:
        print(f'K = {k}, Exp = {exp}')

        list_acc = []
        fold = 1
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            clf = NWKNN(n_neighbors=k, exp=exp)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            #print(clf.weights)
            #print(clf.Data)
            
            #print(pd.value_counts(y_train))
            
            #print(pd.DataFrame(X_train))
            
            #print("X_train")
            #print(X_train)
            ##print("y_train")
            #print(y_train)
            #print("X_test")
            #print(X_test)
            #print("y_test")
            #print(y_test)
            
            acc = accuracy_score(y_test, y_pred)
            print(f'\tFold {fold} = {acc}')
            list_acc.append(acc)
            print(confusion_matrix(y_test, y_pred))
            print(classification_report(y_test, y_pred))
            fold += 1
        
        avg_acc = np.average(list_acc)
        if (avg_acc > best_avg_acc):
            best_avg_acc = avg_acc
            best_k = k
            best_exp = exp

        print(f'\tRata-rata acc (K={k}, exp={exp}):', avg_acc)

print('BEST K: ', best_k)
print('BEST Exp: ', best_exp)
print('BEST Avg. Acc.: ', best_avg_acc)

K = 20, Exp = 4
	Fold 1 = 0.900990099009901
[[11  5  0]
 [ 0 66  2]
 [ 0  3 14]]
              precision    recall  f1-score   support

      Rendah       1.00      0.69      0.81        16
      Sedang       0.89      0.97      0.93        68
      Tinggi       0.88      0.82      0.85        17

    accuracy                           0.90       101
   macro avg       0.92      0.83      0.86       101
weighted avg       0.91      0.90      0.90       101

	Fold 2 = 0.9207920792079208
[[12  4  0]
 [ 0 66  2]
 [ 0  2 15]]
              precision    recall  f1-score   support

      Rendah       1.00      0.75      0.86        16
      Sedang       0.92      0.97      0.94        68
      Tinggi       0.88      0.88      0.88        17

    accuracy                           0.92       101
   macro avg       0.93      0.87      0.89       101
weighted avg       0.92      0.92      0.92       101

	Fold 3 = 0.9306930693069307
[[14  2  0]
 [ 0 68  1]
 [ 0  4 12]]
              precision  

In [6]:
dataHasilEuclidian = np.array(clf.Data[9]).T

In [7]:
print(pd.DataFrame(dataHasilEuclidian))

          0         1         2         3         4         5         6    \
0    3.464102  3.872983  5.744563  3.464102  5.477226  9.000000  5.099020   
1    3.316625  3.162278  7.071068  3.000000  4.582576  7.348469  5.000000   
2    4.123106  4.242641  9.055385  3.316625  5.000000  5.477226  7.141428   
3    2.236068  2.000000  5.099020  3.000000  4.358899  8.000000  4.582576   
4    3.162278  3.000000  7.000000  2.828427  4.242641  6.557439  5.477226   
..        ...       ...       ...       ...       ...       ...       ...   
903  3.605551  4.000000  8.366600  3.605551  5.744563  6.164414  6.244998   
904  4.358899  4.242641  7.745967  4.358899  5.916080  7.483315  6.708204   
905  3.464102  3.316625  8.062258  2.828427  4.690416  5.567764  6.164414   
906  3.162278  3.605551  7.810250  3.464102  4.898979  6.403124  6.324555   
907  3.464102  3.605551  7.810250  3.162278  5.099020  6.244998  6.164414   

          7         8         9    ...       91        92        93   \
0  

In [ ]:
#dataLatihhhh = np.array(clf.y_train).tolist()

In [ ]:
#pd.DataFrame(dataLatihhhh).head(909)

In [ ]:
# Dataset Kolom Kelas
dataset_df['kelas']

In [ ]:
# Jumlah masing masing kelas pada dat latih fold terakhir
pd.value_counts(y_train)

In [ ]:
dataset_df.count

In [ ]:
import pickle

model = NWKNN(n_neighbors=20, exp=4)
model.fit(X_train, y_train)
filehandler = open("test.obj","wb")
pickle.dump(clf,filehandler)
filehandler.close()

with open("test.obj",'rb') as file:
    newClf = pickle.load(file)



In [ ]:
newClf.predict(X_test)

In [ ]:
print(y_test) #fold terakhir

In [ ]:
y_pred #fold terakhir

In [ ]:
y_train.count

In [ ]:
pd.set_option('display.max_rows', None)
y_test

In [ ]:
pd.set_option('display.max_rows', None)
y_pred

In [ ]:
import pickle

model = NWKNN(n_neighbors=20, exp=4)
model.fit(X_train,y_train)
filehandler = open("test_baru.obj","wb")
pickle.dump(clf,filehandler)
filehandler.close()

with open("test_baru.obj",'rb') as file:
    model_ = pickle.load(file)


In [ ]:
import pickle

model = NWKNN(n_neighbors=20, exp=4)

with open("test_baru.obj",'rb') as file:
    model_ = pickle.load(file)

In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample

data_dataset_df = pd.read_csv('dataset_terbaru_fix.csv')
X = data_dataset_df.iloc[:, :22] 
y = data_dataset_df['kelas']

clf = NWKNN(n_neighbors=20, exp=4)
clf.fit(X_train, y_train)

data_baru = pd.read_excel('data_implementasi.xlsx')
X_test = data_baru.iloc[:, :-1].values

y_pred = clf.predict(X_test)

y_pred = le.inverse_transform(y_pred)

# Menampilkan hasil prediksi
print("Hasil prediksi:")
for i in range(len(y_pred)):
    print("Data {}: {}".format(i+1, y_pred[i]))

In [ ]:
with open('test_baru.pickle', 'rb') as file:
    clf_clf = pickle.load(file)

# # Menguji data baru
# data_baru = pd.read_excel('data_implementasi.xlsx')  # Ganti nama_file.xlsx dengan nama file Excel yang berisi data baru
# X_baru = data_baru.values
# prediksi = knn.predict(X_baru)

# # Mengubah label numerik menjadi kategori jika diperlukan
# prediksi_kategori = label_encoder.inverse_transform(prediksi)

# # Menampilkan hasil prediksi
# print("Hasil prediksi:")
# for hasil in prediksi_kategori:
#     print(hasil)

In [ ]:
import os

os.path.isfile('test_baru.obj')

In [ ]:
newClfnew = open("test_baru.obj",'rb')
object_file = pickle.load(newClf)
